In [15]:
import numpy as np
import tools

NSCALE_points = 7
NSCALE_map = np.linspace(0, 3, NSCALE_points)

ZA_points = 7
air_masses_map = np.linspace(1, tools.calc_air_mass(75), ZA_points)
# ZA_map scaling with linear air_mass
ZA_map = tools.calc_zenith(air_masses_map)

freq_points = 240001
Tb_scalar_field = np.zeros((NSCALE_points, ZA_points, freq_points))
NSCALE_jacobian = np.zeros((NSCALE_points, ZA_points, freq_points))
AM_jacobian = np.zeros((NSCALE_points, ZA_points, freq_points))

# Load interpolation & jacobian data
for idx, NSCALE in enumerate(NSCALE_map):
    for jdx, AM in enumerate(air_masses_map):

        filename = f'MaunaKea_Tb_Spectrum_{AM:.1f}_{NSCALE:.1f}'
        data = np.load(f'{filename}.out')

        freq_map = data[:,0]
        
        # Second column is Tb in K
        Tb_scalar_field[idx,jdx] = data[:,2]

        # Jacobian
        conversation_factor = 2109.86 * 10**-3
        NSCALE_jacobian[idx,jdx] = (data[:,4] / conversation_factor)
        AM_jacobian[idx,jdx] = (data[:,3])


In [27]:
from scipy.interpolate import CubicHermiteSpline

for idx, NSCALE in enumerate(NSCALE_map):
    interp_func = CubicHermiteSpline(x=air_masses_map[::2],
                                     y=Tb_scalar_field[idx,::2],
                                     dydx=AM_jacobian[idx,::2])
    
    points = np.meshgrid(air_masses_map)

    eval_grid = interp_func(points).reshape(AM_jacobian[idx].shape)
    


In [24]:
interp_func = CubicHermiteSpline(x=air_masses_map[::2], y=Tb_scalar_field[0,::2],dydx=AM_jacobian[0,::2])

In [26]:
eval_grid.shape

(7, 240001)

In [25]:
Tb_scalar_field[0,::2].shape

(4, 240001)

In [30]:
air_masses_map[::2].shape

(4,)

In [28]:
AM_jacobian[0].shape

(7, 240001)